### Carga de librerías

In [15]:
# Cargar librerías necesarias
import os
os.environ["AZUREML_LOG_DEPRECATION_WARNING_ENABLED"] = "False"
import json
from azureml.core import Workspace, Model, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

### Deployment del modelo

In [ ]:
#### Crear un workspace de Azure
# Cargar mi ID desde variables de entorno
id = os.environ.get("Azure_ID")

# Crear el workspace de Azure
ws = Workspace.create(name="ML_Workspace",
                   subscription_id=id,
                   resource_group='ML_india',
                   location ='centralindia') # centralindia

In [ ]:
# Cargar el workspace de Azure
ws = Workspace.get(name='ML_Workspace', subscription_id=id, resource_group='ML_india', location='centralindia')

Registering model MyModel


In [13]:
ModelName = 'MyModel'
RegisteredModel = Model.register(model_path='model.pkl',model_name=ModelName, workspace=ws)

Registering model MyModel


## Score.py

In [14]:
scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    try:
        # En Azure ML
        model_path = Model.get_model_path(model_name='MyModel')
    except Exception as e:
        # En local
        model_path = os.path.join(os.getcwd(), "model.pkl")

    model = joblib.load(model_path)

def run(data):
    try:
        data = json.loads(data)
        df = pd.DataFrame(data['data'])

        # Verificación y limpieza
        if 'Bankrupt?' in df.columns:
            df = df.drop(columns=['Bankrupt?'])
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])

        # Predicción
        result = model.predict(df).tolist()
        return json.dumps(result)
    except Exception as e:
        return json.dumps({{"error": str(e)}})
"""

# Crear carpeta si no existe
os.makedirs('scripts', exist_ok=True)

# Guardar el archivo en la subcarpeta /scripts
with open('scripts/score.py', 'w') as f:
    f.write(scorepy)


## Configuración del entorno

In [16]:
# Crear un entorno de ejecución personalizado llamado 'env-4-housing'
VirtualEnv = Environment(name='env-4-ML')

# Especificar las dependencias Conda necesarias: scikit-learn y pandas
VirtualEnv.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn', 'pandas', 'numpy', 'xgboost', 'joblib'], pip_packages=['imbalanced-learn'])

# Crear la configuración de inferencia, especificando el entorno y el script de entrada 'score.py'
InterfaceConfig = InferenceConfig(environment=VirtualEnv, entry_script='scripts/score.py')

# Configurar los recursos para el despliegue en un servicio de contenedor de Azure (ACI):
# 0.5 núcleos de CPU y 1 GB de RAM
AciConfig = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

# Desplegar el modelo registrado en el workspace 'ws' como un servicio web llamado 'service',
# utilizando la configuración de inferencia y de ACI definidas anteriormente.
# Si ya existe un servicio con ese nombre, se sobrescribirá.
service = Model.deploy(
    workspace=ws,
    name='service',
    models=[RegisteredModel],
    inference_config=InterfaceConfig,
    deployment_config=AciConfig,
    overwrite=True
)

service.wait_for_deployment(show_output=True)
print("Deployment state:", service.state)
print("Scoring URI:", service.scoring_uri)

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.
C:\Users\aesca\AppData\Local\Temp\ipykernel_15780\3879730266.py:17: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-04 19:32:51-06:00 Creating Container Registry if not exists.
2025-04-04 19:32:52-06:00 Use the existing image.
2025-04-04 19:32:52-06:00 Generating deployment configuration.
2025-04-04 19:32:53-06:00 Submitting deployment to compute.
2025-04-04 19:32:56-06:00 Checking the status of deployment service.
TimedOut


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 41ba5bcf-8c95-4c34-81e9-4de03570646e
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. 
Please refer to https://aka.ms/debugimage#dockerlog for more information."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 41ba5bcf-8c95-4c34-81e9-4de03570646e
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. 
Please refer to https://aka.ms/debugimage#dockerlog for more information."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 41ba5bcf-8c95-4c34-81e9-4de03570646e\nMore information can be fo

## Deployment 😎

In [ ]:
# Mostrar la URL del servicio web desplegado
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"scoring_uri": scoring_uri})

file = open('ScoreUri.json', 'w')
file.write(scoreuri)
file.close()